In [ ]:
# Packages
import pandas as pd
import numpy as np
import geopandas as gpd
import math

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import KFold, GroupKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans

import statsmodels.api as sm
from scipy.spatial import distance

import matplotlib.pyplot as plt
import matplotlib as mpl

import spacv
from spacv.grid_builder import construct_blocks

In [ ]:
# load the dataset for domestic violence prediction

df_DV = pd.read_csv("../Data/DV/DV.csv")
df_DV[["CensusBloc"]] = df_DV[["CensusBloc"]].astype(str)
y = df_DV["DV rate"]
df_DV.head()

In [ ]:
# Create the geodataframe for the data

gdf_DV = gpd.GeoDataFrame(df_DV, geometry=gpd.points_from_xy(df_DV['Lonpro'], df_DV['Latpro']))

In [ ]:
# Define the predictors

using_columns = ['population density', '% White', '% Ame Indi and AK Native', '% Asian', '% Nati Hawa and Paci Island', 
                 '% Hispanic', '% age 18-29', '% age 30-39', '% age 40-49', '% age 50-59', '% age >60', 'med income', 
                 '% unemployment', '% female hh', '% <highschool', '% security inc', '% assistant inc', '% renter hh', 
                 '% stay >=5yrs']
num_features = len(using_columns)
num_features

In [ ]:
# Standardization function

def standarize_data(data, stats):
    return (data - stats['mean'])/stats['std']

## Random CV

In [ ]:
y_predict = []
y_true = []

ten_fold = KFold(n_splits=10, shuffle=True, random_state=42)

i = 1

for train_index, test_index in ten_fold.split(df_DV):
    print("fold:", str(i))

    X_train_all, X_test_all = df_DV.iloc[train_index], df_DV.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    X_train = X_train_all[using_columns]
    X_test = X_test_all[using_columns]
    
    training_stat = X_train.describe().transpose()
    X_scaled_train = standarize_data(X_train, training_stat)
    X_scaled_test = standarize_data(X_test, training_stat)
    
    rf = RandomForestRegressor(n_estimators=80, max_features='sqrt', random_state=42, bootstrap=False)
    rf.fit(X_scaled_train, y_train)
    
    rf_predict = rf.predict(X_scaled_test)    
    y_predict = y_predict + list(rf_predict)
    y_true = y_true + y_test.tolist()
    
    i = i + 1

rmse = mean_squared_error(y_true, y_predict, squared=False)  # False means return RMSE value
r2 = r2_score(y_true, y_predict)
print("rmse: " + str(round(rmse,4)), "r2: " + str(round(r2,4)))

## Clustering-based spatial CV

In [ ]:
# Split the data based on their coordinates using k-means clustering algorithm

coordinates = df_DV[['Lonpro','Latpro']]

kmeans = KMeans(n_clusters=10, random_state=42).fit(coordinates)
centroids = kmeans.cluster_centers_

plt.scatter(coordinates['Lonpro'], coordinates['Latpro'], c= kmeans.labels_.astype(float), s=50, alpha=0.5)
plt.scatter(centroids[:, 0], centroids[:, 1], c='red', s=50)
plt.show()

In [ ]:
# label the cluster index of each sample. 

df_DV_cluster = df_DV.copy()
df_DV_cluster["cluster"] = kmeans.labels_.tolist()
df_DV_cluster["cluster"].value_counts()

In [ ]:
y_predict = []
y_true = []

group_index = df_DV_cluster['cluster'].values

group_kfold = GroupKFold(n_splits=10)

i = 1

for train_index, test_index in group_kfold.split(df_DV_cluster, y, group_index):
    print("fold:", str(i))

    X_train_all, X_test_all = df_DV_cluster.iloc[train_index], df_DV_cluster.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    X_train = X_train_all[using_columns]
    X_test = X_test_all[using_columns]
    
    training_stat = X_train.describe().transpose()
    X_scaled_train = standarize_data(X_train, training_stat)
    X_scaled_test = standarize_data(X_test, training_stat)
    
    rf = RandomForestRegressor(n_estimators=200, max_features='sqrt', random_state=42, bootstrap=False)
    rf.fit(X_scaled_train, y_train)
    
    rf_predict = rf.predict(X_scaled_test)    
    y_predict = y_predict + list(rf_predict)
    y_true = y_true + y_test.tolist()    
    
    i = i + 1

rmse = mean_squared_error(y_true, y_predict, squared=False)  # False means return RMSE value
r2 = r2_score(y_true, y_predict)
print("rmse: " + str(round(rmse,4)), "r2: " + str(round(r2,4)))

## Grid-based spatial CV

In [ ]:
# Split the data using grids

grid_cv = spacv.HBLOCK(3, 3, method='unique', buffer_radius=0).split(gdf_DV['geometry'])

In [ ]:
y_predict = []
y_true = []

i = 1

for train_index, test_index in grid_cv:
    print("fold:", str(i))

    X_train_all, X_test_all = df_DV.iloc[train_index], df_DV.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    X_train = X_train_all[using_columns]
    X_test = X_test_all[using_columns]
    
    training_stat = X_train.describe().transpose()
    X_scaled_train = standarize_data(X_train, training_stat)
    X_scaled_test = standarize_data(X_test, training_stat)
    
    rf = RandomForestRegressor(n_estimators=200, max_features='sqrt', random_state=42, bootstrap=False)
    rf.fit(X_scaled_train, y_train)
    
    rf_predict = rf.predict(X_scaled_test)    
    y_predict = y_predict + list(rf_predict)
    y_true = y_true + y_test.tolist()
        
    i = i + 1

rmse = mean_squared_error(y_true, y_predict, squared=False)  # False means return RMSE value
r2 = r2_score(y_true, y_predict)
print("rmse: " + str(round(rmse,4)), "r2: " + str(round(r2,4)))

## Geo-attribute-based spatial CV

In [ ]:
# Load the CBG-level and neighborhoods-level data of Chicago 

gdf_cbg = gpd.read_file("../Data/DV/chicago_cbg.shp")
gdf_cbg = gdf_cbg.to_crs('epsg:26916')

gdf_neigh = gpd.read_file("../Data/DV/chicago_neighborhoods.shp")
gdf_neigh = gdf_neigh.to_crs('epsg:26916')

In [ ]:
# Determine which neighborhood each CBG is located in based on the intersection area.

fid_list = []

for index1, row1 in gdf_cbg.iterrows():
    geometry1 = row1["geometry"]
    percentage = 0
    fid = 0
    for index2, row2 in gdf_neigh.iterrows():
        geometry2 = row2["geometry"]
        fid_temp = index2
        perc_temp = (geometry1.intersection(geometry2).area/geometry1.area)*100
        if perc_temp > percentage:
            percentage = perc_temp
            fid = fid_temp
    fid_list.append(fid)

gdf_cbg["neigh_id"] = fid_list
gdf_cbg.neigh_id.nunique()

In [ ]:
df_DV_block = df_DV.merge(gdf_cbg[['CensusBloc','neigh_id']], how='left', left_on="CensusBloc", right_on="CensusBloc")
df_DV_block.neigh_id.nunique()

In [ ]:
y_predict = []
y_true = []

block = df_DV_block['neigh_id'].values
group_kfold = GroupKFold(n_splits=96)

i = 1

for train_index, test_index in group_kfold.split(df_DV, y, block):
    print("fold:", str(i))

    X_train_all, X_test_all = df_DV.iloc[train_index], df_DV.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    X_train = X_train_all[using_columns]
    X_test = X_test_all[using_columns]
    
    training_stat = X_train.describe().transpose()
    X_scaled_train = standarize_data(X_train, training_stat)
    X_scaled_test = standarize_data(X_test, training_stat)
    
    rf = RandomForestRegressor(n_estimators=200, max_features='sqrt', random_state=42, bootstrap=False) 
    rf.fit(X_scaled_train, y_train)
    
    rf_predict = rf.predict(X_scaled_test)    
    y_predict = y_predict + list(rf_predict)
    y_true = y_true + y_test.tolist()    
        
    i = i + 1

rmse = mean_squared_error(y_true, y_predict, squared=False)  # False means return RMSE value
r2 = r2_score(y_true, y_predict)
print("rmse: " + str(round(rmse,4)), "r2: " + str(round(r2,4)))

## Spatial leave-one-out CV

In [ ]:
# Compute the radius of buffer as the 0.05 quantile of distances of data

from itertools import combinations

lng_lat_coords = np.array(df_DV[['Lonpro','Latpro']])

distances = [distance.euclidean(p1, p2) for p1, p2 in combinations(lng_lat_coords, 2)]
distances_array=np.array(distances)
np.quantile(distances_array, 0.05)

In [ ]:
# Split the training and test data for each fold using the buffer_radius

skcv = spacv.SKCV(n_splits=2146, buffer_radius=3004, random_state=42).split(gdf_DV['geometry'])

In [ ]:
y_predict = []
y_true = []

i = 0

for train_index, test_index in skcv:
    print("fold:", str(i))
    
    X_train_all, X_test_all = df_DV.iloc[train_index], df_DV.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    X_train = X_train_all[using_columns]
    X_test = X_test_all[using_columns]
    
    training_stat = X_train.describe().transpose()
    X_scaled_train = standarize_data(X_train, training_stat)
    X_scaled_test = standarize_data(X_test, training_stat)
    
    rf = RandomForestRegressor(n_estimators=80, max_features='sqrt', random_state=42, bootstrap=False) 
    rf.fit(X_scaled_train, y_train)
    
    rf_predict = rf.predict(X_scaled_test)    
    y_predict = y_predict + list(rf_predict)
    y_true = y_true + y_test.tolist()    
    
    i = i + 1

rmse = mean_squared_error(y_true, y_predict, squared=False)  # False means return RMSE value
r2 = r2_score(y_true, y_predict)
print("rmse: " + str(round(rmse,4)), "r2: " + str(round(r2,4)))